<a href="https://colab.research.google.com/github/sokrypton/7.571/blob/main/L4/FDR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multiple Hypothesis Testing and False Discovery Rate

In this notebook, we'll explore:
1. The multiple testing problem (why p < 0.05 isn't enough)
2. How p-values are distributed under null vs alternative
3. Bonferroni correction (too conservative)
4. Benjamini-Hochberg procedure for controlling False Discovery Rate (FDR)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

## 1 | The Multiple Testing Problem

**Scenario:** You measure expression of 1000 genes in control vs treatment.

If you use p < 0.05 as your threshold, how many false positives do you expect?

Let's simulate a case where **NO genes are actually affected** (all null hypotheses are true).

In [ ]:
# Simulate 1000 genes with NO real effect
num_genes = 1000
n_samples = 10  # samples per group

p_values = []
for gene in range(num_genes):
    # Both groups come from the SAME distribution (no effect)
    control = np.random.normal(100, 20, n_samples)
    treatment = np.random.normal(100, 20, n_samples)
    _, p = ttest_ind(control, treatment)
    p_values.append(p)

p_values = np.array(p_values)

# How many "significant" results?
false_positives = np.sum(p_values < 0.05)

print(f"Tested {num_genes} genes (all with NO real effect)")
print(f"Found {false_positives} 'significant' results (p < 0.05)")
print(f"\nThese are ALL false positives!")
print(f"Expected: {num_genes * 0.05:.0f} (5% of {num_genes})")

### Key insight

With 1000 tests at α = 0.05, we expect **~50 false positives** even when nothing is real!

This is the **multiple testing problem**.

---

## 2 | Distribution of p-values

The distribution of p-values tells us a lot:

In [ ]:
# When ALL null hypotheses are true, p-values are UNIFORM
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.hist(p_values, bins=20, edgecolor='white', alpha=0.7)
plt.axhline(num_genes/20, color='red', linestyle='--', label='Expected if uniform')
plt.xlabel('p-value')
plt.ylabel('Count')
plt.title('All nulls TRUE → p-values are UNIFORM')
plt.legend()

# Now simulate with SOME real effects
num_null = 900      # genes with no effect
num_real = 100      # genes with real effect

p_values_mixed = []

# Null genes (no effect)
for gene in range(num_null):
    control = np.random.normal(100, 20, n_samples)
    treatment = np.random.normal(100, 20, n_samples)  # same mean
    _, p = ttest_ind(control, treatment)
    p_values_mixed.append(p)

# Real effect genes
for gene in range(num_real):
    control = np.random.normal(100, 20, n_samples)
    treatment = np.random.normal(130, 20, n_samples)  # different mean!
    _, p = ttest_ind(control, treatment)
    p_values_mixed.append(p)

p_values_mixed = np.array(p_values_mixed)

plt.subplot(1, 2, 2)
plt.hist(p_values_mixed, bins=20, edgecolor='white', alpha=0.7)
plt.axhline(num_genes/20, color='red', linestyle='--', label='Expected if all null')
plt.xlabel('p-value')
plt.ylabel('Count')
plt.title('Some effects REAL → spike near 0')
plt.legend()

plt.tight_layout()
plt.show()

print("Left: All null true → flat (uniform) distribution")
print("Right: Some real effects → spike at low p-values")

### Understanding the histogram

- **Flat part** = null genes (p-values are uniformly distributed)
- **Spike near 0** = real effects (low p-values) + some false positives

The challenge: **How do we separate real effects from false positives?**

---

## 3 | Correction Methods

### Bonferroni Correction

The simplest approach: divide α by the number of tests.

$$\text{Bonferroni threshold} = \frac{\alpha}{m} = \frac{0.05}{1000} = 0.00005$$

**Problem:** This is very conservative - we miss many real effects!

In [ ]:
# We know the truth (first 900 are null, last 100 are real)
is_null = np.array([True]*num_null + [False]*num_real)

# No correction
threshold_none = 0.05
discoveries_none = p_values_mixed < threshold_none

# Bonferroni
threshold_bonf = 0.05 / len(p_values_mixed)
discoveries_bonf = p_values_mixed < threshold_bonf

print("Comparison of thresholds:\n")
print(f"No correction:  p < {threshold_none}")
print(f"Bonferroni:     p < {threshold_bonf:.2e}\n")

print(f"{'Method':<20} {'Threshold':<15} {'Discoveries':<12} {'True Pos':<10} {'False Pos':<10}")
print("-" * 67)

tp_none = np.sum(discoveries_none & ~is_null)
fp_none = np.sum(discoveries_none & is_null)
print(f"{'No correction':<20} {'p < 0.05':<15} {np.sum(discoveries_none):<12} {tp_none:<10} {fp_none:<10}")

tp_bonf = np.sum(discoveries_bonf & ~is_null)
fp_bonf = np.sum(discoveries_bonf & is_null)
print(f"{'Bonferroni':<20} {f'p < {threshold_bonf:.1e}':<15} {np.sum(discoveries_bonf):<12} {tp_bonf:<10} {fp_bonf:<10}")

print(f"\nBonferroni is too conservative!")
print(f"We have {num_real} real effects, but Bonferroni only finds {tp_bonf}.")

---

## 4 | False Discovery Rate (FDR)

Instead of controlling the **false positive rate** (probability of any false positive), we control the **False Discovery Rate**:

$$\text{FDR} = \frac{\text{False Positives}}{\text{All Discoveries}}$$

If we make 100 discoveries and FDR = 0.05, we expect ~5 of them to be false.

### Benjamini-Hochberg Procedure

**Algorithm:**
1. Sort p-values from smallest to largest: P₁ ≤ P₂ ≤ ... ≤ Pₘ
2. Find the largest j where Pⱼ (p-value at rank j) ≤ j × α/m
3. Reject all hypotheses with p-value ≤ Pⱼ

In [ ]:
def benjamini_hochberg(p_values, alpha=0.05):
    """Apply BH procedure, return boolean array of rejections and cutoff"""
    m = len(p_values)

    # Step 1: Sort p-values (P_1 <= P_2 <= ... <= P_m)
    sorted_indices = np.argsort(p_values)
    sorted_pvals = p_values[sorted_indices]

    # Step 2: Find largest j where P_j <= j * alpha / m
    ranks = np.arange(1, m + 1)  # j = 1, 2, ..., m
    thresholds = ranks * alpha / m  # j * alpha / m
    below_threshold = sorted_pvals <= thresholds

    if not any(below_threshold):
        return np.zeros(m, dtype=bool), 0

    # Find j: the largest rank where P_j <= j * alpha / m
    j = np.max(np.where(below_threshold)[0]) + 1  # +1 because ranks start at 1
    P_j = sorted_pvals[j - 1]  # P_j is the cutoff

    # Step 3: Reject all hypotheses with p-value <= P_j
    rejected = p_values <= P_j

    return rejected, P_j

# Apply BH procedure
alpha = 0.05
rejected_bh, P_j = benjamini_hochberg(p_values_mixed, alpha=alpha)

tp_bh = np.sum(rejected_bh & ~is_null)
fp_bh = np.sum(rejected_bh & is_null)
total_bh = np.sum(rejected_bh)

print(f"Benjamini-Hochberg (α = {alpha}):")
print(f"  Cutoff P_j = {P_j:.4f}")
print(f"  Total discoveries: {total_bh}")
print(f"  True positives: {tp_bh}")
print(f"  False positives: {fp_bh}")
if total_bh > 0:
    print(f"  Actual FDR = {fp_bh}/{total_bh} = {fp_bh/total_bh:.1%}")

### Why does BH work? Visualizing sorted p-values

**Key insight:** When all nulls are true, sorted p-values follow a diagonal line (slope ≈ 1/m). The BH threshold line has a much smaller slope (α/m). Real effects pull p-values **below** the BH line.

In [ ]:
# Sort BOTH sets of p-values for comparison
sorted_null = np.sort(p_values)  # all null case from earlier
sorted_mixed = np.sort(p_values_mixed)  # some real effects

m = num_genes
ranks = np.arange(1, m + 1)

# BH threshold line: j * alpha / m
alpha = 0.05
bh_line = ranks * alpha / m

# Expected p-values under null: ~j/m (diagonal)
expected_null = ranks / m

In [ ]:
# Side-by-side comparison: Full view
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: ALL NULL
ax = axes[0]
ax.plot(ranks, sorted_null, 'b.', markersize=3, alpha=0.6, label='Sorted p-values')
ax.plot(ranks, bh_line, 'r-', linewidth=2, label='BH threshold (j × α/m)')
ax.plot(ranks, expected_null, 'k--', linewidth=1, alpha=0.5, label='Expected under null (j/m)')
ax.set_xlabel('Rank (j)')
ax.set_ylabel('p-value')
ax.set_title('ALL NULL (no real effects)\n→ p-values stay ABOVE BH line')
ax.legend(loc='upper left')
ax.set_xlim(0, m)
ax.set_ylim(0, 1)
ax.grid(True, alpha=0.3)

# Right: SOME REAL
ax = axes[1]
ax.plot(ranks, sorted_mixed, 'b.', markersize=3, alpha=0.6, label='Sorted p-values')
ax.plot(ranks, bh_line, 'r-', linewidth=2, label='BH threshold (j × α/m)')
ax.plot(ranks, expected_null, 'k--', linewidth=1, alpha=0.5, label='Expected under null (j/m)')
ax.set_xlabel('Rank (j)')
ax.set_ylabel('p-value')
ax.set_title(f'SOME REAL ({num_real} real effects)\n→ p-values dip BELOW BH line')
ax.legend(loc='upper left')
ax.set_xlim(0, m)
ax.set_ylim(0, 1)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Side-by-side comparison: Zoomed view
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
zoom = 150

# Left: ALL NULL (zoomed)
ax = axes[0]
ax.plot(ranks[:zoom], sorted_null[:zoom], 'bo-', markersize=4, alpha=0.7, label='Sorted p-values')
ax.plot(ranks[:zoom], bh_line[:zoom], 'r-', linewidth=2, label='BH threshold')
ax.plot(ranks[:zoom], expected_null[:zoom], 'k--', linewidth=1, alpha=0.5, label='Expected (j/m)')
ax.set_xlabel('Rank (j)')
ax.set_ylabel('p-value')
ax.set_title('ALL NULL (zoomed)\n→ p-values stay ABOVE red line')
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

# Right: SOME REAL (zoomed)
ax = axes[1]
ax.plot(ranks[:zoom], sorted_mixed[:zoom], 'bo-', markersize=4, alpha=0.7, label='Sorted p-values')
ax.plot(ranks[:zoom], bh_line[:zoom], 'r-', linewidth=2, label='BH threshold')
ax.plot(ranks[:zoom], expected_null[:zoom], 'k--', linewidth=1, alpha=0.5, label='Expected (j/m)')

# Find and mark the cutoff
below_bh = sorted_mixed <= bh_line
if any(below_bh):
    j_cutoff = np.max(np.where(below_bh)[0]) + 1
    ax.axvline(j_cutoff, color='green', linestyle=':', linewidth=2, label=f'Cutoff (j={j_cutoff})')

ax.set_xlabel('Rank (j)')
ax.set_ylabel('p-value')
ax.set_title('SOME REAL (zoomed)\n→ real effects dip BELOW red line')
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Left: All null - p-values follow diagonal (j/m), stay ABOVE BH line → no discoveries")
print(f"Right: Some real - p-values dip BELOW BH line → discoveries!")

### Visualizing BH vs Bonferroni

In [ ]:
# Sort p-values
sorted_pvals = np.sort(p_values_mixed)
m = len(sorted_pvals)
ranks = np.arange(1, m + 1)

# BH threshold line (diagonal)
fdr_level = 0.05
bh_line = ranks / m * fdr_level

# Bonferroni threshold (horizontal)
bonf_threshold = 0.05 / m

# Find cutoffs
below_bh = sorted_pvals <= bh_line
cutoff_rank_bh = np.max(np.where(below_bh)[0]) + 1 if any(below_bh) else 0
cutoff_rank_bonf = np.sum(sorted_pvals <= bonf_threshold)

# Plot
plt.figure(figsize=(10, 6))

# Plot sorted p-values
plt.plot(ranks, sorted_pvals, 'b.', markersize=3, alpha=0.5, label='p-values (sorted)')

# Plot BH threshold line (diagonal)
plt.plot(ranks, bh_line, 'g-', linewidth=2, label=f'BH threshold (FDR = {fdr_level})')

# Plot Bonferroni threshold (horizontal)
plt.axhline(bonf_threshold, color='purple', linestyle='--', linewidth=2,
            label=f'Bonferroni (α/m = {bonf_threshold:.1e})')

# Mark cutoffs
if cutoff_rank_bh > 0:
    plt.axvline(cutoff_rank_bh, color='green', linestyle=':', alpha=0.7)
if cutoff_rank_bonf > 0:
    plt.axvline(cutoff_rank_bonf, color='purple', linestyle=':', alpha=0.7)

plt.xlabel('Rank (sorted by p-value)', fontsize=12)
plt.ylabel('p-value', fontsize=12)
plt.title('BH vs Bonferroni: BH allows more discoveries', fontsize=14)
plt.legend(loc='lower right')
plt.xlim(0, m)
plt.ylim(0, 1)
plt.grid(True, alpha=0.3)
plt.show()

print(f"BH discovers {cutoff_rank_bh} genes (p-values below green line)")
print(f"Bonferroni discovers {cutoff_rank_bonf} genes (p-values below purple line)")

### Zooming in to see the difference

In [ ]:
# Zoom in to first 150 ranks
plt.figure(figsize=(10, 6))
zoom = 150

plt.plot(ranks[:zoom], sorted_pvals[:zoom], 'bo-', markersize=5, alpha=0.7, label='p-values')
plt.plot(ranks[:zoom], bh_line[:zoom], 'g-', linewidth=2, label='BH threshold')
plt.axhline(bonf_threshold, color='purple', linestyle='--', linewidth=2, label='Bonferroni threshold')

# Mark where BH and Bonferroni cut off
if cutoff_rank_bh > 0 and cutoff_rank_bh < zoom:
    plt.axvline(cutoff_rank_bh, color='green', linestyle=':', linewidth=2,
                label=f'BH cutoff (rank {cutoff_rank_bh})')
if cutoff_rank_bonf > 0 and cutoff_rank_bonf < zoom:
    plt.axvline(cutoff_rank_bonf, color='purple', linestyle=':', linewidth=2,
                label=f'Bonferroni cutoff (rank {cutoff_rank_bonf})')

plt.xlabel('Rank', fontsize=12)
plt.ylabel('p-value', fontsize=12)
plt.title('Zoomed view: BH adapts threshold based on rank', fontsize=14)
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)
plt.show()

print("Key insight:")
print("  - Bonferroni uses a fixed, very low threshold (purple dashed line)")
print("  - BH uses an adaptive threshold that increases with rank (green line)")
print("  - This allows BH to make more discoveries while controlling FDR")

---

## 5 | Final Comparison

In [ ]:
print(f"Comparison ({num_real} real effects out of {num_genes} genes):\n")
print(f"{'Method':<25} {'Discoveries':<12} {'True Pos':<10} {'False Pos':<10} {'FDR':<10}")
print("-" * 67)

# No correction
disc = np.sum(p_values_mixed < 0.05)
tp = np.sum((p_values_mixed < 0.05) & ~is_null)
fp = np.sum((p_values_mixed < 0.05) & is_null)
fdr = fp / disc if disc > 0 else 0
print(f"{'p < 0.05 (no correction)':<25} {disc:<12} {tp:<10} {fp:<10} {fdr:<10.1%}")

# Bonferroni
disc = np.sum(p_values_mixed < bonf_threshold)
tp = np.sum((p_values_mixed < bonf_threshold) & ~is_null)
fp = np.sum((p_values_mixed < bonf_threshold) & is_null)
fdr = fp / disc if disc > 0 else 0
print(f"{'Bonferroni':<25} {disc:<12} {tp:<10} {fp:<10} {fdr:<10.1%}")

# BH
disc = np.sum(rejected_bh)
tp = np.sum(rejected_bh & ~is_null)
fp = np.sum(rejected_bh & is_null)
fdr = fp / disc if disc > 0 else 0
print(f"{'Benjamini-Hochberg':<25} {disc:<12} {tp:<10} {fp:<10} {fdr:<10.1%}")

print("\n" + "="*67)
print("No correction: Many false positives (high FDR)")
print("Bonferroni:    Very few false positives, but misses most real effects")
print("BH:            Controls FDR while detecting more real effects!")

---

## Summary

### The Problem
- Testing many hypotheses → many false positives by chance
- With 1000 tests at α = 0.05, expect ~50 false positives

### P-value Distribution
- **All nulls true** → p-values are uniform (flat histogram)
- **Some real effects** → spike near 0 + flat background

### Key Visual Insight
- **Sorted p-values under null:** follow diagonal line (slope ≈ 1/m)
- **BH threshold line:** much flatter (slope = α/m)
- **Real effects:** pull p-values DOWN, below the BH line → discoveries!

### Correction Methods

| Method | Threshold | Controls | Problem |
|--------|-----------|----------|--------|
| **None** | p < 0.05 | Nothing | Too many false positives |
| **Bonferroni** | p < α/m | Family-wise error rate | Too conservative |
| **Benjamini-Hochberg** | Adaptive | False Discovery Rate | Best balance! |

### Benjamini-Hochberg Procedure
1. Sort p-values: P₁ ≤ P₂ ≤ ... ≤ Pₘ
2. Find the largest j where Pⱼ ≤ j × α/m
3. Reject all hypotheses with p-value ≤ Pⱼ

**Expected FDR ≤ α** (on average across many experiments)